# Streaming RNN-T for Engineer

Khi cần huấn luyện một mô hình RNN-T (Recurrent Neural Network Transducer), nhiều người sẽ nghĩ ngay đến việc sử dụng các framework như NeMo hoặc k2 vì được hỗ trợ rất tốt và cung cấp nhiều mô hình sẵn có để thử nghiệm. Tuy nhiên, các framework này thường rất tổng quát, phức tạp và khó tùy chỉnh theo nhu cầu riêng. Gần đây có rất nhiều các pretrained audio được chia sẻ mà các framework này chưa hỗ trợ, việc tích hợp chúng vào các framework lớn có thể mất khá nhiều thời gian nếu không quen sử dụng. Trong bài viết này, mình sẽ xây dựng lại 1 mô hình RNN-T từ đầu sao cho dễ hiểu và dễ tinh chỉnh nhất có thể, làm cho quá trình thử nghiệm nhanh chóng hơn mà kết quả không thua gì các framework phức tạp. 

Bài viết sẽ đi từ: Lý thuyết RNN-T -> Xây dựng lần lượt Encoder, Decoder, Jointer -> Quá trình streaming sau khi train -> Tối ưu tốc độ với ONNX

Tài liệu tham khảo: 
- [Sequence-to-sequence learning with Transducers bởi Loren Lugosch](https://lorenlugosch.github.io/posts/2020/11/transducer/)
- Nhiều phần code sẽ được mượn từ [NeMo](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/asr)
- Model được lấy thẳng từ [repo Whisper](https://github.com/openai/whisper/blob/main/whisper/model.py) của openai

# Cấu trúc tổng quan của RNN-T

<video src="/wp/videos/RNNTVisualization.mp4" width="900" height="600" controls></video>

RNN-T gồm 3 thành phần chính:

- Encoder là một mạng bất kỳ, thường nhận đầu vào là âm thanh dạng `(batch, time)` hoặc mel-spectrogram có dạng `(batch, num_bins, time)`, sinh ra một ma trận biểu diễn $\bf f$ của âm thanh với dạng `(batch, time_resampling, hidden_dim)`.

- Decoder là 1 mạng RNN, nhận đầu vào là chuỗi ký tự với dạng `(batch, time_char)` và 1 hoặc nhiều state dạng `(batch, 1, hidden_dim)`. Ở mỗi bước sinh ra một ma trận biểu diễn $\bf  g$ của chuỗi ký tự với dạng `(batch, 1, hidden_dim)` và 1 (GRU) hoặc nhiều state mới (LSTM) dạng `(batch, 1, hidden_dim)`.

- Jointer: Kết hợp biểu diễn từ Encoder và Decoder để tạo ra biểu diễn cuối cùng $h_{t,u}$. Phân phối xác suất ký tự tiếp theo được tính toán thông qua một lớp Softmax.


Chúng ta sẽ xây dựng theo thứ tự: Feature extractor, Encoder, Decoder, Jointer. Cụ thể, phần Encoder ta sẽ tận dụng encoder của whisper-small, phần Decoder ta sẽ dùng GRU, phần Jointer là một mạng MLP.

Một vài constant được dùng trong bài:

```python
# Sample rate of the audio
SAMPLE_RATE = 16000 

# Parameters for the STFT
N_FFT = 400
HOP_LENGTH = 160
N_MELS = 80

# Parameters for the RNN-T
RNNT_BLANK = 1024
PAD = 1 # tokenizer.pad_id()
VOCAB_SIZE = 1024

# Parameters for the mask
ATTENTION_CONTEXT_SIZE = (80, 3)
```

# Các bước triển khai

## Mel-spectrogram

Trong lúc training, quy trình trích xuất mel-spectrogram từ âm thanh sẽ được thực hiện giống hệt Whisper gốc, chỉ bỏ đi phần chuẩn hóa (normalization) do ta không có thông tin về max khi streaming. 

Trong lúc streaming, ta để `pad_center=False` thay vì `True` như lúc training để tránh các chunk bị pad.

```python
# Training
stft = torch.stft(audio, N_FFT, HOP_LENGTH, window=window, return_complex=True)

# Streaming
stft = torch.stft(audio, N_FFT, HOP_LENGTH, window=window, center=False, return_complex=True)
```

Đầu vào: `audio` với dạng `(batch, time)`

Đầu ra: `mel` với dạng `(batch, num_bins, time)`

## Encoder

### Cấu trúc của encoder

Encoder của RNN-T thường có 2 phần:
1. Lớp sub-sampling: giảm số lượng frame của âm thanh, thường là các lớp Convolution liên tiếp
2. Lớp Transformer/RNN

Cụ thể trong Whisper: 
1. Subsampling: 2 lớp Conv1D  ([code]((https://github.com/openai/whisper/blob/main/whisper/model.py#L179))), subsampling rate là 2 do chỉ có 1 Conv1D có stride=2

```python
        self.conv1 = Conv1d(n_mels, n_state, kernel_size=3, padding=1)
        self.conv2 = Conv1d(n_state, n_state, kernel_size=3, stride=2, padding=1)
```

2. Transformer/RNN: nhiều lớp transformers

Chúng ta sẽ thay đổi cấu trúc của Whisper encoder một chút để có subsampling rate cao hơn bản gốc (âm thanh được nén nhiều hơn). Cụ thể:
1. Subsampling: 3 lớp 1D CNN; Cả 3 lớp đều có stride 2, kernel size 3, padding 0. Như vậy subsampling rate là 8.

```python
        self.conv1 = Conv1d(n_mels, n_state, kernel_size, padding=0, stride=2)
        self.conv2 = Conv1d(n_state, n_state, kernel_size, padding=0, stride=2)
        self.conv3 = Conv1d(n_state, n_state, kernel_size, padding=0, stride=2)
```
\* *Ở đây mình dùng toàn bộ Conv1d không có padding cho đơn giản phần inference, tuy nhiên nó có thể làm giảm chất lượng lúc inference streaming. Lý do là khi inference, ta cần caching một vài trạng thái và khởi tạo bằng 0, model không được train với các trạng thái bằng 0 đó nên kết quả có thể sai khác lúc inference offline. Phần subsampling này có thể thiết kế lại khéo hơn để đảm bảo đồng bộ trong cả trường hợp offline và online.*

2. Transformer/RNN: Giữ nguyên, thêm masking cho ma trận attention weight và KV cache

### Thay thế absolute positional encoding

Trong cấu trúc của Whisper, toàn bộ âm thanh sau khi được subsample sẽ được [cộng với ma trận absolute positional encoding](https://github.com/openai/whisper/blob/main/whisper/model.py#L198). Phần này phải được thay thế để phù hợp cho streaming do Whisper chỉ hỗ trợ âm thanh với độ dài tối đa 30s nên ma trận positional encoding cũng chỉ được tạo để hỗ trợ độ dài tối đa 30s. Nếu âm thanh streaming vượt quá 30s, model sẽ không hiểu được.

Chúng ta sẽ thử nghiệm ALiBi thay thế cho Absolute Positional Encoding (Ngoài ra có thể dùng Relative Positional Encoding của Transformers XL, RoPE, ...). Một lợi điểm của ALiBi khi sử dụng trong bài toán này là trong lúc streaming, ta có thể thay đổi số lượng chunk quá khứ có thể nhìn vào mà không quá ảnh hưởng đến chất lượng của model.

Implementation gốc: https://github.com/ofirpress/attention_with_linear_biases

Implement trong NeMo: https://github.com/NVIDIA/NeMo/blob/cef98dbaa61971b889bb2484916b90c11a4c2a2d/nemo/collections/nlp/modules/common/megatron/position_embedding/alibi_relative_position_embedding.py#L41

Implementation của ALiBi đơn giản, chỉ cần thêm bias vào ma trận scores trước khi đưa vào softmax. Mình sử dụng hàm `scaled_dot_product_attention` của torch nên cần sửa như dưới đây:

```python
mask = mask + alibi_mask

a = scaled_dot_product_attention(
    q, k, v, attn_mask=mask, is_causal=False # is_causal must be False since we prepare the mask ourselves
)
```

### Streaming transformers

Với Attention, mỗi timestep sẽ được tính toán attention với tất cả các timestep khác. Để streaming, ta cần thay đổi sao cho mỗi timestep chỉ được tính toán attention với các timestep trong quá khứ và có thể một vài timestep trong tương lai. Để hạn chế phải lưu trữ quá nhiều thông tin quá khứ, ta cũng nên giới hạn số timestep quá khứ mà timestep hiện tại có thể dùng. Điều này có thể giải quyết bằng masking.

Trong bài này, ta sẽ dụng phương pháp Chunk-aware look-ahead theo paper: https://arxiv.org/pdf/2312.17279

- Chunk-aware look-ahead chia các frame sau subsampling thành các chunk, cho phép frame trong chunk hiện tại có thể nhìn thấy các frame trong các chunk trước và chunk hiện tại. Điều này giúp model có thể tận dụng được cả thông tin quá khứ và một chút ở tương lai, đảm bảo tốc độ streaming.
- Với mỗi timestep, chúng sẽ chỉ được tính toán attention với các timestep trong chunk hiện tại và một vài chunk phía trước nó, các giá trị khác đều được mask. Tất cả các lớp Transformers đều sử dụng chung 1 mask này.

Một video ví dụ đơn giản về chunk-aware look-ahead với `ATTENTION_CONTEXT_SIZE = (4, 1)`, nghĩa là chunk size = 2, look ahead 1 chunk, history 2 chunk (4 frames).

<video src="/wp/videos/AttentionMask.mp4" width="900" height="600" controls></video>

Trong bài này, với config `ATTENTION_CONTEXT_SIZE = (80, 3)` ta sẽ dùng:
- Chunk size: 4
- Look back (history): 20 chunks
- Look ahead: 1 chunk

Code tạo mask được tận dụng từ NeMo [ở đây](https://github.com/NVIDIA/NeMo/blob/b1dd398904e85a630cba50dfe302223e47943750/nemo/collections/asr/modules/conformer_encoder.py#L702). Mask này sẽ được kết hợp với mask của ALiBi như đã ghi chú ở trên.

## Decoder & Jointer

- [Decoder GRU](link to code)
- [Jointer MLP](link to code)

## Caching

To read: https://kipp.ly/transformer-inference-arithmetic/

Caching cần thiết cho streaming khi ta đánh đổi không gian lưu trữ với tốc độ tính toán.

### Transformers KV cache

Trong quá trình inference, ta chỉ có chunk hiện tại và một vài chunk phía trước nó. Với cơ chế attention không có KV cache, ta phải tính toán lại key và value của tất cả các frame phía trước. Việc lưu trữ key và value của các frame phía trước sẽ giúp giảm thời gian inference. Tuy nhiên nếu GPU của bạn nhanh thì KV cache không cần thiết, vì không giống LLM, model của chúng ta lúc nào chỉ cần tính toán lại cố định key và value của 80 timestep trước.

[Code ở đây]

### Conv1D cache

Để đảm bảo inference lúc streaming y hệt như inference lúc có toàn bộ audio, các lớp Conv1D cần caching lại kết quả của một vài trạng thái trước. Cụ thể trong trường hợp của chúng ta, mỗi lớp Conv1D cần caching lại kết quả của 1 trạng thái trước.

Cụ thể trong trường hợp của chúng ta với 3 lớp Conv1D như trên:
- conv1: cache 1 + 31 frame mới
- conv2: cache 1 + 16 frame mới
- conv3: cache 1 + 8 frame mới

<video src="/wp/videos/CNNStreaming.mp4" width="900" height="600" controls></video>

Như minh họa ở video trên, nếu ta có toàn bộ mel-spectrogram từ 0->64 , ta có kết quả của subsampling là tensor từ 0 -> 7. Nếu ta thực hiện chunking với caching như trên, ta sẽ có kết quả từ 0->3 và từ 4->7, ghép lại y hệt như khi ta có đầy đủ mel-spectrogram.

## Quá trình streaming

Tóm tắt lại, sau quá trình training, để model có thể stream được ta cần:

- Khởi tạo caching (kv cache, conv1d cache, hidden_state của RNN)
- Chỉnh sửa STFT phần trích xuất mel-spectrogram `pad_center = False`

Đến đây, ta có thể đưa các chunk vào model, cập nhật cache và cứ lặp lại như vậy đến hết âm thanh.

## ONNX export & quantization

### Mel-spectrogram

Phần trích xuất mel-spectrogram, ta sẽ sử dụng code ở đây để có thể export ra ONNX. Chỉ cần chú ý là `pad_center` phải là `False` do ta đang streaming. Source [github](https://github.com/echocatzh/conv-stft)

### Encoder

[code export ONNX]

Có một vài warning về việc có 1 vài tensor có thể trở thành constant. Tuy nhiên phần này không ảnh hưởng, do khi inference các tensor này sẽ không thay đổi, ta có thể bỏ qua warning này.

### Decoder & Jointer

Việc export decoder và jointer ra ONNX dễ dàng hơn, không có warning gì.

### Quantization

Cuối cùng, để model nhỏ hơn nữa và chạy nhanh trên CPU, ta có thể quantize model. Ở đây ta sẽ sử dụng dynamic quantization của ONNX với weight_type là QInt8.

[code ở đây]

# Performance

| Model config | Device |Size (MB) | Dataset | WER (%) |
| --- | --- | --- | --- | --- |
| FP32 + Online | GPU | 300 | VIVOS | 14.38 |
| FP32 + Offline | GPU | 300 | VIVOS | 14.19 |
| INT4 + Online | CPU | 100 | VIVOS | 21.51 |
| FP32 + Online | GPU | 300 | CommonVoice 17 vi | 8.92 |
| FP32 + Offline | GPU | 300 | CommonVoice 17 vi | 14.52 |
| INT4 + Online | CPU | 100 | CommonVoice 17 vi | 28.52 |

Model của chúng ta còn cách ra xa với các model CTC hay các model RNN-T chất lượng cao khác. Ở trong bộ Commonvoice 17 Vietnamese ta còn thấy có sự khác nhau khá lớn giữa Online và Offline, khả năng cao là do model bị overfit vào domain của bộ này. Công việc tiếp theo chắc chắn sẽ là lọc và thu thập thêm data. Một vài nhược điểm của model có thể nhận thấy như model thường xuyên bỏ qua một vài chữ, WER cao với giọng vùng miền khó, inference online hơi khác offline. Tuy nhiên nó đã hoàn thành mục tiêu chính là cung cấp cái nhìn sâu hơn về cách hoạt động của RNN-T.